In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2025-02-17 08:51:34.466128: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-17 08:51:34.544997: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739807494.565780   65447 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739807494.571427   65447 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-17 08:51:34.595745: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [15]:
def callCLM(params,ix,path):
    # initialize sample with default CLM parameter set
    sample = np.array([0.8737026 , 0.14630213, 0.37171737, 0.27391456, 0.46666708,
       0.51924798, 0.46736367, 0.68843046, 0.61951923, 0.07068093,
       0.23517169, 0.17849673, 0.83990038, 0.10616273, 0.44688065,
       0.57367004, 0.30371528, 0.91975309, 0.71671051, 0.51094701,
       0.61565899, 0.26983746, 0.93698915, 0.02468359, 0.05703913,
       0.61071442, 0.461464  , 0.11014087, 0.71658162, 0.99813995,
       0.19665675, 0.71452042])

    sample[ix] = params
    
    loaded_emulator = tf.saved_model.load(path)
    y_pred, y_pred_var = loaded_emulator.predict(sample.reshape(1,-1))
    
    return y_pred.numpy().flatten()

In [16]:
# get LHC PPE parameter sets 
lhckey = './utils/lhc220926.txt'
ppe_params = pd.read_csv(lhckey).drop(columns='member')

In [17]:
tuning_params = ['FUN_fracfixers','froot_leaf','jmaxb0','jmaxb1','kmax','leafcn','medlynintercept','medlynslope','slatop','wc2wjb0']

In [20]:
num_params = len(tuning_params)
params = np.random.rand(1, num_params)
ix = [ppe_params.columns.get_loc(col) for col in tuning_params]

emulator_path = './emulators/laimax_gmean/'
laimax = callCLM(params,ix,emulator_path) # m2/m2 (annual maximum LAI)

emulator_path = './emulators/gpp_gmean/'
gpp = callCLM(params,ix,emulator_path) # PgC/year (annual mean GPP)

emulator_path = './emulators/biomass_gmean/'
biomass = callCLM(params,ix,emulator_path) # PgC (annual mean biomass)